In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

duckduckgo = DuckDuckGoSearchRun( description="Use this to search any questions or queries asked by user that need to use search engine")

for chunk in duckduckgo.stream("what is the current weather in salem, tamilnadu, india"):
    print(chunk)

Salem weather today shows a temperature of 17.7°C with Clear conditions. The current weather in Salem feels like 17.7°C, with humidity at 92%, wind blowing at 6.8 km/h, and a UV Index of 0. Visibility stands at 10 km. According to the latest Salem weather update, sunrise was at 07:46 AM and sunset is expected at 05:00 PM. Get the latest hourly weather updates for Salem today. Detailed forecast including temperature, wind, rain, snow, and UV index. Stay informed about today's weather conditions in Salem . Salem , Tamil Nadu, Weather Forecast with current conditions, temperature, precipitation and wind for the next 14 days. How's the weather today in Salem , Tamil Nadu. We provide information on: wind speed, wind direction, atmospheric pressure, temperature, humidity and visibility. Current weather condition is Clear with real-time temperature (22°C), humidity 58%, wind 7.6km/h, pressure (1012mb), UV (0), visibility (10km) in Tamil Nadu.


In [2]:
from langchain.tools import tool

@tool
def send_email(text : str) -> str:
    """Send emails. Use ONLY when user asks to send an email/gmail."""
    return "Email sent"

@tool(return_direct=True)
def get_datetime() -> str:
    """Get current date and time. Use when user asks for current date/time."""
    from datetime import datetime
    return datetime.now().strftime("%d %B %Y, %I:%M %p")

In [ ]:

import os
# from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()
os.getenv("GROQ_API_KEY")
from langchain.chat_models import init_chat_model

llm = init_chat_model("groq:llama-3.1-8b-instant")

llm.invoke("hey")

AIMessage(content='Hello.  Is there something I can help you with, or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 36, 'total_tokens': 56, 'completion_time': 0.046639882, 'completion_tokens_details': None, 'prompt_time': 0.002507294, 'prompt_tokens_details': None, 'queue_time': 0.051122495, 'total_time': 0.049147176}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bdbd5-2567-78d1-bab0-8c3cf8c3853d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 36, 'output_tokens': 20, 'total_tokens': 56})

In [38]:
# from langchain.agents import create_agent
import requests
load_dotenv()

token = os.getenv("ACCESS_TOKEN_WHATSAPPP")
@tool
def send_whatsapp_messages(to : str, message: str) -> str:
    
    """Send a WhatsApp TEXT message. 
Inputs: to (phone number), message (text body)."""
    
    print(to, message)
    try:
        url = "https://graph.facebook.com/v22.0/962571180271822/messages"
        data = {
  "messaging_product": "whatsapp",
  "to": to,
  "type": "template",
  "template": {
    "name": "hello_world",
    "language": { "code": "en_US" }                                                                 
  }                                                                 
}                                                                                                           

        res = requests.post(url=url, headers={
            "Authorization" : f"Bearer {token}",
            "Content-Type": "application/json"
        }, json=data)
        
        print("PAYLOAD:", data) 
        
        if res.status_code == 200:
            return f"✅ Message sent to {to}"
        else:
            return f"❌ Failed: {res.status_code} | {res.text}"
        
        # return f"Message send to {to}"
    
    except Exception as e:
        return f"Error in sending message {e}"
        
    
    
    
    

In [39]:
toolKit = [duckduckgo, send_email, get_datetime, send_whatsapp_messages]
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=toolKit,
    
)

In [40]:
example_query = "send whatsapp message to 919566765097 sending the message as 'Hello there from agent'"

for mode, event in agent.stream(
    {"messages": [("user", example_query)]},
    stream_mode=["updates"],
):
    print(mode, event)
# # print(events)
# for event in events:
#     print(event["messages"].content)

updates {'model': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'gbyt3bmt2', 'function': {'arguments': '{"message":"Hello there from agent","to":"919566765097"}', 'name': 'send_whatsapp_messages'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 480, 'total_tokens': 509, 'completion_time': 0.03827947, 'completion_tokens_details': None, 'prompt_time': 0.074335631, 'prompt_tokens_details': None, 'queue_time': 0.055316938, 'total_time': 0.112615101}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bdc08-a996-7832-afaa-b196d7d4056b-0', tool_calls=[{'name': 'send_whatsapp_messages', 'args': {'message': 'Hello there from agent', 'to': '919566765097'}, 'id': 'gbyt3bmt2', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 480, 'output_toke